In [1]:
from pymongo import MongoClient
import pandas as pd
import json as js
import os
import requests

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [9]:
test = (db["cleanOffices"].aggregate([
   {
     '$geoNear': {
        'near': { 'type': "Point", 'coordinates': [ -73.996876 , 40.724604 ] },
        'distanceField': "dist.calculated",
        'maxDistance': 100,
        'query': { 'city': "New York" },
        'spherical': 'true'
     }
   }
]))

a = list(test)
a

[{'_id': ObjectId('5e415340c99528f967a9b6c8'),
  'name': 'AddThis',
  'category_code': 'advertising',
  'description': 'New York Office',
  'address1': '568 Broadway',
  'address2': '11th Floor, Suite 1105',
  'zip_code': '10012',
  'city': 'New York',
  'state_code': 'NY',
  'country_code': 'USA',
  'latitude': 40.724604,
  'longitude': -73.996876,
  'location': {'type': 'Point', 'coordinates': [-73.996876, 40.724604]},
  'dist': {'calculated': 0.0}},
 {'_id': ObjectId('5e415340c99528f967a9b773'),
  'name': 'appssavvy',
  'category_code': 'advertising',
  'description': 'Headquarters',
  'address1': '594 Broadway',
  'address2': 'Suite 207',
  'zip_code': '10012',
  'city': 'New York',
  'state_code': 'NY',
  'country_code': 'USA',
  'latitude': 40.724604,
  'longitude': -73.996876,
  'location': {'type': 'Point', 'coordinates': [-73.996876, 40.724604]},
  'dist': {'calculated': 0.0}},
 {'_id': ObjectId('5e415340c99528f967a9b78d'),
  'name': 'AppNexus',
  'category_code': 'advertising

In [4]:
# Filter offices. Just testing, my friend!
officesF = pd.DataFrame(list(db['cleanOffices'].find({'city': {'$in': ['New York', 'San Francisco', 'Mountain View']}})))

In [5]:
comp = (db['oldestOffices'].aggregate([{'$geoNear': {'near': { 'type': "Point", 'coordinates': [-73.996876, 40.724604] },
        'distanceField': "dist.calculated",
        'maxDistance': 2000,
        'query': { 'city': 'New York' },
        'spherical': 'true'}}]))
                                                              
print(len(list(comp)))                                              

36


In [6]:
def officesComp(coord, bd, city):
    '''Function to know distance from office to a offices founded before 2003'''
    coordinates = coord['coordinates']
    comp = (db[f'{bd}'].aggregate([{'$geoNear': {'near': { 'type': "Point", 'coordinates': coordinates },
        'distanceField': "dist.calculated",
        'maxDistance': 2000,
        'query': { 'city': f'{city}' },
        'spherical': 'true'}}]))
    officescomp = len(list(comp))
    return officescomp

In [7]:
officesF['competitors NY'] = officesF['location'].apply(lambda x: officesComp(x, 'oldestOffices', 'New York'))
officesF['competitors SF'] = officesF['location'].apply(lambda x: officesComp(x, 'oldestOffices','San Francico'))
officesF['competitors MV'] = officesF['location'].apply(lambda x: officesComp(x, 'oldestOffices','Mountain View'))
# Fatla incluir airports porque da un error al importar a Mongodb

KeyError: 'location'

In [92]:
officesF.head()
# ¿No hay oficinas en SF y MV?

,_id,name,category_code,description,address1,address2,zip_code,city,state_code,country_code,location,competitors NY,competitors SF,competitors MV
0,5e4003ab35e23c13dc2b242c,AddThis,advertising,New York Office,568 Broadway,"11th Floor, Suite 1105",10012,New York,NY,USA,"{'type': 'Point', 'coordinates': [-73.996876, ...",36,0,0
1,5e4003ab35e23c13dc2b242d,Kyte,games_video,None,442 Post Street,10th Floor,94102,San Francisco,CA,USA,"{'type': 'Point', 'coordinates': [-122.409173,...",0,0,0
2,5e4003ab35e23c13dc2b242e,Joost,games_video,,100 5th Ave Fl 6,,10011-6903,New York,NY,USA,"{'type': 'Point', 'coordinates': [-74.0094471,...",34,0,0
3,5e4003ab35e23c13dc2b242f,Jingle Networks,mobile,,475 Park Ave South,10th Floor,10016,New York,NY,USA,"{'type': 'Point', 'coordinates': [-122.173887,...",0,0,0
4,5e4003ab35e23c13dc2b2430,Jajah,mobile,Jajah HQ,2513 Charleston Road,Suite 102,94043,Mountain View,CA,USA,"{'type': 'Point', 'coordinates': [-122.0899512...",0,0,5


In [106]:

"""
# Etiquetas
airport
train_station
bar"""


# Google Maps requests
def requestMap(lat, long, tipo):
    token = os.getenv('MAPS_KEY')
    url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?key={token}&location={lat},{long}&type={tipo}'
    res = requests.get(url)
    if res != 200:
        raise ValueError('Bad Response')
    return res.json()

In [75]:
officesF['airports'] = officesF['location'].apply(lambda x: requestMap(x, tipo='airport'))


ValueError: Bad Response

In [ ]:
sort values